In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

from warnings import simplefilter
simplefilter('ignore')

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples"]
start_date = "2019-09-01"
end_date = "2019-10-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names').drop_duplicates()
issues = issues.query('author not in @bot_names').drop_duplicates()
prs = prs.query('author not in @bot_names').drop_duplicates()

In [6]:
# Only keep the datapoints we want
query_data = 'updatedAt > @start_date and updatedAt < @end_date and org in @github_orgs'
comments = comments.query(query_data)
issues = issues.query(query_data)
prs = prs.query(query_data)

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/ipython](https://github.com/ipython)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyterlab](https://github.com/jupyterlab)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-09-01** to **2019-10-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 30 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                      repo  author
  38       jupyterlab                jupyterlab       5
  42       jupyterlab            jupyterlab-git       5
  41       jupyterlab  jupyterlab-data-explorer       4
  36       jupyterlab                  debugger       3
  45       jupyterlab              team-compass       2
  37       jupyterlab         jupyter-renderers       1
  39       jupyterlab       jupyterlab-celltags       1
  40       jupyterlab     jupyterlab-commenting       1
  43       jupyterlab     jupyterlab-shortcutui       1
  44       jupyterlab      jupyterlab-telemetry       1
  27       jupyterhub                jupyterhub      16
  30       jupyterhub       mybinder.org-deploy      13
  28       jupyterhub   jupyterhub-the-hard-way       8
  34       jupyterhub             traefik-proxy       5
  35       jupyterhub    zero-to-jupyterhub-k8s       5
  25       jupyterhub              batchspawner       3
  33       jupyterhub   the-littlest-jupyterhub       3
  31       jupyterhub               nbgitpuller       2
  26       jupyterhub      jupyter-server-proxy       1
  29       jupyterhub               kubespawner       1
  32       jupyterhub              team-compass       1
  22  jupyter-widgets                ipyleaflet       4
  23  jupyter-widgets        jupyterlab-sidecar       1
  24  jupyter-widgets    widget-ts-cookiecutter       1
  10          jupyter              jupyter-book      45
  14          jupyter            jupyter_server      20
  7           jupyter        enterprise_gateway       4
  12          jupyter            jupyter_client       3
  6           jupyter             docker-stacks       2
  13          jupyter              jupyter_core       2
  19          jupyter                 qtconsole       2
  8           jupyter                governance       1
  9           jupyter                   jupyter       1
  11          jupyter            jupyter-sphinx       1
  15          jupyter            kernel_gateway       1
  16          jupyter                  nbgrader       1
  17          jupyter                  nbviewer       1
  18          jupyter                  notebook       1
  20          jupyter               repo2docker       1
  21          jupyter                 telemetry       1
  3           ipython                 ipykernel       5
  4           ipython               ipyparallel       3
  5           ipython                   ipython       1
  1   binder-examples       multi-language-demo       2
  0   binder-examples         jupyter-extension       1
  2   binder-examples                         r       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['jupyterlab', 'jupyterlab-git', 'jupyterlab-data-explorer', 'debugger', 'team-compass', 'jupyter-renderers', 'jupyterlab-celltags', 'jupyterlab-commenting', 'jupyterlab-shortcutui', 'jupyterlab-telemetry', 'jupyterhub', 'mybinder.org-deploy', 'jupyterhub-the-hard-way', 'traefik-proxy', 'zero-to-jupyterhub-k8s', 'batchspawner', 'the-littlest-jupyterhub', 'nbgitpuller', 'jupyter-server-proxy', 'kubespawner', 'team-compass', 'ipyleaflet', 'jupyterlab-sidecar', 'widget-ts-cookiecutter', 'jupyter-book', 'jupyter_server', 'enterprise_gateway', 'jupyter_client', 'docker-stacks', 'jupyter_core', 'qtconsole', 'governance', 'jupyter', 'jupyter-sphinx', 'kernel_gateway', 'nbgrader', 'nbviewer', 'notebook', 'repo2docker', 'telemetry', 'ipykernel', 'ipyparallel', 'ipython', 'multi-language-demo', 'jupyter-extension', 'r'],
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merged PRs in the last 30 days'
})

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:         username  authored authorAssociation
    0         NHDaly         1       CONTRIBUTOR
    1           fm75         1       CONTRIBUTOR
    2         hdrake         1       CONTRIBUTOR
    3  pablobernabeu         1       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:         username  authored authorAssociation
    5          minrk         3            MEMBER
    3       impact27         2       CONTRIBUTOR
    0        Chilipp         1       CONTRIBUTOR
    1  edisongustavo         1       CONTRIBUTOR
    2         hugovk         1       CONTRIBUTOR
    4     mdickinson         1       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 30 days',
    width: 1000
  }), Chart({
    data:           username  authored authorAssociation
    9        choldgraf        32            MEMBER
    4          Zsailer        12            MEMBER
    3         SamLau95         7      COLLABORATOR
    15     kevin-bates         6            MEMBER
    22         vidartf         3      COLLABORATOR
    11         emdupre         3      COLLABORATOR
    1            MSeal         3            MEMBER
    12        impact27         2       CONTRIBUTOR
    23        willingc         2            MEMBER
    10       dkhokhlov         2              NONE
    16        krinsman         1       CONTRIBUTOR
    21     victorsamun         1              NONE
    20     psychemedia         1       CONTRIBUTOR
    19    playermanny2         1       CONTRIBUTOR
    18         pfmoore         1       CONTRIBUTOR
    17         parente         1            MEMBER
    0            BertR         1            MEMBER
    14      jasongrout         1            MEMBER
    13          ivanov         1            MEMBER
    8          betatim         1            MEMBER
    7           arnavs         1       CONTRIBUTOR
    6         akhmerov         1            MEMBER
    5     akhilputhiry         1              NONE
    2   MichelleUfford         1       CONTRIBUTOR
    24       yuvipanda         1            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
    

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org                 repo   username  closed
    0  binder-examples    jupyter-extension  choldgraf       1
    1  binder-examples  multi-language-demo    betatim       2
    2  binder-examples                    r  choldgraf       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:        org         repo   username  closed
    3  ipython    ipykernel  blink1073       5
    4  ipython  ipyparallel      minrk       3
    5  ipython      ipython    Carreau       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 30 days',
    width: 1000
  }), Chart({
    data:         org                repo     username  closed
    6   jupyter       docker-stacks      parente       2
    7   jupyter  enterprise_gateway  kevin-bates       3
    8   jupyter  enterprise_gateway     lresende       1
    9   jupyter          governance       ivanov       1
    10  jupyter             jupyter    choldgraf       1
    11  jupyter        jupyter-book     SamLau95       1
    12  jupyter        jupyter-book    choldgraf      44
    13  jupyter      jupyter-sphinx     akhmerov       1
    14  jupyter      jupyter_client        MSeal       1
    15  jupyter      jupyter_client    blink1073       1
    16  jupyter      jupyter_client       rgbkrk       1
    17  jupyter        jupyter_core        minrk       2
    18  jupyter      jupyter_server      Zsailer       4
    19  jupyter      jupyter_server  kevin-bates      11
    20  jupyter      jupyter_server     rolweber       4
    21  jupyter      jupyter_server     willingc       1
    22  jupyter      kernel_gateway  kevin-bates       1
    23  jupyter            nbgrader     willingc       1
    24  jupyter            nbviewer      parente       1
    25  jupyter            notebook     lresende       1
    26  jupyter           qtconsole   ccordoba12       2
    27  jupyter         repo2docker      betatim       1
    28  jupyter           telemetry      Zsailer       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 30 days',
    width: 1000
  }), Chart({
    data:                 org                    repo       username  closed
    29  jupyter-widgets              ipyleaflet    martinRenou       4
    30  jupyter-widgets      jupyterlab-sidecar  SylvainCorlay       1
    31  jupyter-widgets  widget-ts-cookiecutter        vidartf       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter-widgets in the last 30 days',
    width: 1000
  }), Chart({
    data:            org                     repo        username  closed
    32  jupyterhub             batchspawner        willingc       3
    33  jupyterhub     jupyter-server-proxy         betatim       1
    34  jupyterhub               jupyterhub  GeorgianaElena       1
    35  jupyterhub               jupyterhub    consideRatio       1
    36  jupyterhub               jupyterhub           minrk      12
    37  jupyterhub               jupyterhub        willingc       2
    38  jupyterhub  jupyterhub-the-hard-way        willingc       8
    39  jupyterhub              kubespawner         betatim       1
    40  jupyterhub      mybinder.org-deploy        

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                      repo  number  \
  0           ipython                 ipykernel       4   
  1           ipython               ipyparallel       2   
  2           ipython                   ipython      12   
  3           ipython                 traitlets       2   
  4           jupyter                dashboards       1   
  5           jupyter             docker-stacks       2   
  6           jupyter        enterprise_gateway       4   
  7           jupyter                   jupyter       1   
  8           jupyter              jupyter-book       6   
  9           jupyter            jupyter-sphinx       2   
  10          jupyter            jupyter_client       4   
  11          jupyter            jupyter_server       5   
  12          jupyter                 nbconvert       5   
  13          jupyter                    nbdime       1   
  14          jupyter                  nbgrader       8   
  15          jupyter                  notebook      44   
  16          jupyter                 qtconsole       2   
  17          jupyter               repo2docker       6   
  18  jupyter-widgets                ipyleaflet       2   
  19  jupyter-widgets                ipywidgets      11   
  20  jupyter-widgets        jupyterlab-sidecar       1   
  21  jupyter-widgets                 pythreejs       1   
  22       jupyterhub              batchspawner       2   
  23       jupyterhub                 binderhub       9   
  24       jupyterhub             dockerspawner       2   
  25       jupyterhub     firstuseauthenticator       1   
  26       jupyterhub    jupyter-rsession-proxy       2   
  27       jupyterhub      jupyter-server-proxy       6   
  28       jupyterhub                jupyterhub      10   
  29       jupyterhub  jupyterhub-deploy-docker       1   
  30       jupyterhub               kubespawner       2   
  31       jupyterhub         ldapauthenticator       2   
  32       jupyterhub       mybinder.org-deploy       3   
  33       jupyterhub               nbgitpuller       1   
  34       jupyterhub            oauthenticator       2   
  35       jupyterhub            systemdspawner       1   
  36       jupyterhub              team-compass       2   
  37       jupyterhub   the-littlest-jupyterhub       1   
  38       jupyterhub               wrapspawner       1   
  39       jupyterhub    zero-to-jupyterhub-k8s      23   
  40       jupyterlab                  debugger       2   
  41       jupyterlab                jupyterlab      35   
  42       jupyterlab       jupyterlab-celltags       1   
  43       jupyterlab  jupyterlab-data-explorer       3   
  44       jupyterlab         jupyterlab-github       2   
  45       jupyterlab   jupyterlab-google-drive       1   
  46       jupyterlab            jupyterlab-toc       2   
  47       jupyterlab             pull-requests       1   
  48       jupyterlab              team-compass       1   
  
                                 org/repo  
  0                     ipython/ipykernel  
  1                   ipython/ipyparallel  
  2                       ipython/ipython  
  3                     ipython/traitlets  
  4                    jupyter/dashboards  
  5                 jupyter/docker-stacks  
  6            jupyter/enterprise_gateway  
  7                       jupyter/jupyter  
  8                  jupyter/jupyter-book  
  9                jupyter/jupyter-sphinx  
  10               jupyter/jupyter_client  
  11               jupyter/jupyter_server  
  12                    jupyter/nbconvert  
  13                       jupyter/nbdime  
  14                     jupyter/nbgrader  
  15                     jupyter/notebook  
  16                    jupyter/qtconsole  
  17                  jupyter/repo2docker  
  18           jupyter-widgets/ipyleaflet  
  19           jupyter-widgets/ipywidgets  
  20   jupyter-widgets/jupyterlab-sidecar  
  21            jupyter-widgets/pythreejs  
  22              jup

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                      repo  number  \
  0   binder-examples         jupyter-extension       1   
  1   binder-examples                         r       2   
  2   binder-examples              requirements       1   
  3           ipython                 ipykernel       2   
  4           ipython               ipyparallel       1   
  5           ipython                   ipython      11   
  6           jupyter             docker-stacks       4   
  7           jupyter        enterprise_gateway       3   
  8           jupyter                   jupyter       1   
  9           jupyter              jupyter-book      25   
  10          jupyter            jupyter-sphinx       3   
  11          jupyter            jupyter_client       1   
  12          jupyter            jupyter_server       5   
  13          jupyter            kernel_gateway       1   
  14          jupyter                 nbconvert       5   
  15          jupyter                  nbgrader       6   
  16          jupyter                  nbviewer       1   
  17          jupyter                  notebook      14   
  18          jupyter                 qtconsole       5   
  19          jupyter               repo2docker       5   
  20          jupyter                  steering       1   
  21          jupyter                 telemetry       1   
  22  jupyter-widgets                ipyleaflet       1   
  23  jupyter-widgets                ipywidgets      10   
  24  jupyter-widgets        jupyterlab-sidecar       1   
  25       jupyterhub              batchspawner       5   
  26       jupyterhub                 binderhub      13   
  27       jupyterhub   configurable-http-proxy       1   
  28       jupyterhub             dockerspawner       1   
  29       jupyterhub                jupyterhub      17   
  30       jupyterhub  jupyterhub-deploy-docker       2   
  31       jupyterhub               kubespawner       3   
  32       jupyterhub       mybinder.org-deploy       5   
  33       jupyterhub               nbgitpuller       1   
  34       jupyterhub              team-compass       3   
  35       jupyterhub   the-littlest-jupyterhub       7   
  36       jupyterhub             traefik-proxy       5   
  37       jupyterhub               wrapspawner       1   
  38       jupyterhub    zero-to-jupyterhub-k8s      16   
  39       jupyterlab                  debugger       1   
  40       jupyterlab         jupyter-renderers       1   
  41       jupyterlab                jupyterlab      34   
  42       jupyterlab  jupyterlab-data-explorer       4   
  43       jupyterlab            jupyterlab-git       4   
  44       jupyterlab         jupyterlab-github       1   
  45       jupyterlab   jupyterlab-google-drive       4   
  46       jupyterlab     jupyterlab-shortcutui       1   
  47       jupyterlab      jupyterlab-telemetry       1   
  48       jupyterlab            jupyterlab-toc       1   
  49       jupyterlab              team-compass       2   
  
                                 org/repo  
  0     binder-examples/jupyter-extension  
  1                     binder-examples/r  
  2          binder-examples/requirements  
  3                     ipython/ipykernel  
  4                   ipython/ipyparallel  
  5                       ipython/ipython  
  6                 jupyter/docker-stacks  
  7            jupyter/enterprise_gateway  
  8                       jupyter/jupyter  
  9                  jupyter/jupyter-book  
  10               jupyter/jupyter-sphinx  
  11               jupyter/jupyter_client  
  12               jupyter/jupyter_server  
  13               jupyter/kernel_gateway  
  14                    jupyter/nbconvert  
  15                     jupyter/nbgrader  
  16                     jupyter/nbviewer  
  17                     jupyter/notebook  
  18                    jupyter/qtconsole  
  19                  jupyter/repo2docker  
  20                     jupyter/steering  
  21  

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                  org               repo     kind  count
    49   binder-examples  jupyter-extension  created    0.0
    50   binder-examples                  r  created    0.0
    51   binder-examples       requirements  created    0.0
    111  binder-examples  jupyter-extension   closed    1.0
    112  binder-examples                  r   closed    2.0
    113  binder-examples       requirements   closed    1.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for binder-examples',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:         org         repo     kind  count
    0   ipython    ipykernel  created    4.0
    1   ipython  ipyparallel  created    2.0
    2   ipython      ipython  created   12.0
    3   ipython    traitlets  created    2.0
    62  ipython    ipykernel   closed    2.0
    63  ipython  ipyparallel   closed    1.0
    64  ipython      ipython   closed   11.0
    65  ipython    traitlets   closed    0.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for ipython',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:         org                repo     kind  count
    5   jupyter       docker-stacks  created    2.0
    6   jupyter  enterprise_gateway  created    4.0
    8   jupyter        jupyter-book  created    6.0
    9   jupyter      jupyter-sphinx  created    2.0
    11  jupyter      jupyter_server  created    5.0
    12  jupyter           nbconvert  created    5.0
    14  jupyter            nbgrader  created    8.0
    15  jupyter            notebook  created   44.0
    16  jupyter           qtconsole  created    2.0
    17  jupyter         repo2docker  created    6.0
    67  jupyter       docker-stacks   closed    4.0
    68  jupyter  enterprise_gateway   closed    3.0
    70  jupyter        jupyter-book   closed   25.0
    71  jupyter      jupyter-sphinx   closed    3.0
    73  jupyter      jupyter_server   closed    5.0
    74  jupyter           nbconvert   closed    5.0
    76  jupyter            nbgrader   closed    6.0
    77  jupyter            notebook   closed   14.0
    78  jupyter           qtconsole   closed    5.0
    79  jupyter         repo2docker   closed    5.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 30 days for jupyter',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                      repo   time_open  \
  0   binder-examples         jupyter-extension   14.362072   
  1   binder-examples                         r  469.722853   
  2   binder-examples              requirements   10.026667   
  3           ipython                 ipykernel    1.615538   
  4           ipython               ipyparallel  263.120023   
  5           ipython                   ipython    2.630509   
  6           jupyter             docker-stacks   10.755694   
  7           jupyter        enterprise_gateway    5.193785   
  8           jupyter                   jupyter    4.274792   
  9           jupyter              jupyter-book    4.384780   
  10          jupyter            jupyter-sphinx    0.450694   
  11          jupyter            jupyter_client    1.064676   
  12          jupyter            jupyter_server   76.826562   
  13          jupyter            kernel_gateway    6.249595   
  14          jupyter                 nbconvert   22.170880   
  15          jupyter                  nbgrader    0.859363   
  16          jupyter                  nbviewer    0.024560   
  17          jupyter                  notebook   17.178825   
  18          jupyter                 qtconsole    0.871852   
  19          jupyter               repo2docker  123.679109   
  20          jupyter                  steering  138.787755   
  21          jupyter                 telemetry   53.942755   
  22  jupyter-widgets                ipyleaflet    0.922859   
  23  jupyter-widgets                ipywidgets    6.823675   
  24  jupyter-widgets        jupyterlab-sidecar    0.011343   
  25       jupyterhub              batchspawner    3.289583   
  26       jupyterhub                 binderhub   25.972060   
  27       jupyterhub   configurable-http-proxy  148.413623   
  28       jupyterhub             dockerspawner   27.719259   
  29       jupyterhub                jupyterhub   15.030544   
  30       jupyterhub  jupyterhub-deploy-docker    5.858953   
  31       jupyterhub               kubespawner    0.057894   
  32       jupyterhub       mybinder.org-deploy    4.013715   
  33       jupyterhub               nbgitpuller  111.933079   
  34       jupyterhub              team-compass   66.593183   
  35       jupyterhub   the-littlest-jupyterhub    8.840509   
  36       jupyterhub             traefik-proxy    6.797257   
  37       jupyterhub               wrapspawner  473.767905   
  38       jupyterhub    zero-to-jupyterhub-k8s   21.753981   
  39       jupyterlab                  debugger    4.007581   
  40       jupyterlab         jupyter-renderers   64.790625   
  41       jupyterlab                jupyterlab   46.239786   
  42       jupyterlab  jupyterlab-data-explorer   14.670926   
  43       jupyterlab            jupyterlab-git   13.606557   
  44       jupyterlab         jupyterlab-github    0.010856   
  45       jupyterlab   jupyterlab-google-drive   40.268993   
  46       jupyterlab     jupyterlab-shortcutui  323.841632   
  47       jupyterlab      jupyterlab-telemetry   93.991563   
  48       jupyterlab            jupyterlab-toc    4.644005   
  49       jupyterlab              team-compass   42.873843   
  
                                 org/repo  
  0     binder-examples/jupyter-extension  
  1                     binder-examples/r  
  2          binder-examples/requirements  
  3                     ipython/ipykernel  
  4                   ipython/ipyparallel  
  5                       ipython/ipython  
  6                 jupyter/docker-stacks  
  7            jupyter/enterprise_gateway  
  8                       jupyter/jupyter  
  9                  jupyter/jupyter-book  
  10               jupyter/jupyter-sphinx  
  11               jupyter/jupyter_client  
  12               jupyter/jupyter_server  
  13               jupyter/kernel_gateway  
  14                    jupyter/nbconvert  
  15                     jupyter/nbgrader  
  16                  

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 30 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             author  count authorAssociation
    3        choldgraf     10            MEMBER
    1           NHDaly      7       CONTRIBUTOR
    2          betatim      4            MEMBER
    4             fm75      2       CONTRIBUTOR
    6  matthewfeickert      2              NONE
    0      KirstieJane      1              NONE
    5          marskar      1              NONE
    7    pablobernabeu      1       CONTRIBUTOR
    8            sje30      1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:              author  count authorAssociation
    33         impact27     17       CONTRIBUTOR
    25       ccordoba12     15            MEMBER
    6           Carreau      9            MEMBER
    23        blink1073      8            MEMBER
    42            minrk      6            MEMBER
    48         rmorshea      5      COLLABORATOR
    30    edisongustavo      5       CONTRIBUTOR
    20         asmeurer      4       CONTRIBUTOR
    16    SylvainCorlay      4            MEMBER
    54        tacaswell      3              NONE
    32           histed      3              NONE
    8         GNSharmaa      3              NONE
    44           ndeeds      3              NONE
    46    pauldmccarthy      2              NONE
    11   LucianaMarques      2       CONTRIBUTOR
    28           daharn      2       CONTRIBUTOR
    39       mdickinson      2       CONTRIBUTOR
    10         Khushu06      2              NONE
    38      kevin-bates      2       CONTRIBUTOR
    55       telamonian      2              NONE
    22      bernhard-42      2              NONE
    40            mfh92      1       CONTRIBUTOR
    41       mhsekhavat      1       CONTRIBUTOR
    0            0anton      1              NONE
    43     miss1997yuan      1              NONE
    45   patricknraanes      1              NONE
    36        jhayes-py      1              NONE
    47     reesehopkins      1              NONE
    49        royreznik      1              NONE
    50  schneiderfelipe      1              NONE
    51      shivendra90      1              NONE
    52    stevenleeS0ht      1              NONE
    53           sun-te      1              NONE
    56         vainaijr      1              NONE
    37         katsar0v      1              NONE
    29            dobos      1              NONE
    35       jeffheaton      1              NONE
    34    isaacovercast      1              NONE
    2         0xfeedbac      1              NONE
    3           Aref-cs      1              NONE
    4          BLaurent      1              NONE
    5          BrenBarn      1              NONE
    7        DrAmygdala      1              NONE
    9    ItaloPerez2019      1              NONE
    12        Notgnoshi      1              NONE
    13       QinHuannet      1              NONE
    14        Rafiqul10      1              NONE
    15       Stannislav      1              NONE
    17          Zsailer      1              NONE
    18       alexmojaki      1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
         

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:         author  n_first_responses authorAssociation
    3    choldgraf                  3            MEMBER
    1       NHDaly                  2       CONTRIBUTOR
    0  KirstieJane                  1              NONE
    2      betatim                  1            MEMBER
    4      marskar                  1              NONE
    5        sje30                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 30 days',
    width: 1000
  }), Chart({
    data:              author  n_first_responses authorAssociation
    5           Carreau                  6            MEMBER
    32            minrk                  5            MEMBER
    20       ccordoba12                  4            MEMBER
    18        blink1073                  4            MEMBER
    25           histed                  2              NONE
    40        tacaswell                  2       CONTRIBUTOR
    26         impact27                  2              NONE
    30       mdickinson                  2       CONTRIBUTOR
    0            0anton                  1              NONE
    31       mhsekhavat                  1       CONTRIBUTOR
    27       jeffheaton                  1              NONE
    28        jhayes-py                  1              NONE
    29      kevin-bates                  1       CONTRIBUTOR
    33   patricknraanes                  1              NONE
    23    edisongustavo                  1       CONTRIBUTOR
    34     reesehopkins                  1              NONE
    35         rmorshea                  1      COLLABORATOR
    36  schneiderfelipe                  1              NONE
    37      shivendra90                  1              NONE
    38    stevenleeS0ht                  1              NONE
    39           sun-te                  1              NONE
    41       telamonian                  1              NONE
    24         eranhirs                  1              NONE
    21         cerodell                  1              NONE
    22           daharn                  1       CONTRIBUTOR
    10        Notgnoshi                  1              NONE
    2         0xfeedbac                  1              NONE
    3          BLaurent                  1              NONE
    4          BrenBarn                  1              NONE
    6        DrAmygdala                  1              NONE
    7    ItaloPerez2019                  1              NONE
    8          Khushu06                  1              NONE
    9    LucianaMarques                  1       CONTRIBUTOR
    11       QinHuannet                  1              NONE
    1              0x0L                  1              NONE
    12        Rafiqul10                  1              NONE
    13       Stannislav                  1              NONE
    14       alexmojaki                  1              NONE
    15       archenroot                  1              NONE
    16       bergtholdt                  1       CONTRIBUTOR
    17      bernhard-42                  1              NONE
    19             cc-a                  1              NONE
    42       xenohunter                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'author

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>